In [2]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('songdata.csv')
df.head(3)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...


In [5]:
df.shape

(57650, 4)

In [6]:
df = df.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [7]:
df.shape

(5000, 3)

In [8]:
df['text'] = df['text'].str.lower().replace(r'[^\w\s]','').replace(r'\n',' ', regex=True)

In [9]:
df['text'][0]

"at 4 o'clock we land   and i'll wave my flag for you   interest in heart   we are not savage   like children   we're people   what difference i say      music for chameleons   music for chameleons      a dream of siam   we could leave in the morning   old friends and shy   we need to win   like you need to breathe   i've an interest in games      music for chameleons   music for chameleons      gowns for another show   maybe no-one will come      here take my arm   i like the pictures   and you like ghost stories   tomorrow i'll sigh      music for chameleons   music for chameleons  "

In [18]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [21]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/english.pickle[0m

  Searched in:
    - 'C:\\Users\\dell/nltk_data'
    - 'c:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python310\\nltk_data'
    - 'c:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python310\\share\\nltk_data'
    - 'c:\\Users\\dell\\AppData\\Local\\Programs\\Python\\Python310\\lib\\nltk_data'
    - 'C:\\Users\\dell\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

ModuleNotFoundError: No module named 'sklearn'

In [14]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [16]:
similarity[0]

array([1.        , 0.00527499, 0.        , ..., 0.00607819, 0.        ,
       0.        ])

In [19]:
df[df['song']==''].index[0]

0

# recommedation function

In [39]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [40]:
recommendation('Alma Mater')

['O Love That Will Not Let Me Go',
 'Shall We Dance',
 'Churchill Speech',
 'Until Next Time',
 'The Lord Is My Sheperd',
 'God Bless The Child',
 'Mary Of Argyle',
 'O Little Town Of Bethlehem',
 'A Call To Harvest',
 'Christmas Secrets',
 'The Time Is Near',
 "All Tomorrow's Parties",
 'Fare Thee Well',
 'Dark Star',
 'As With Gladness Men Of Old',
 'As Much As Always',
 'From The Balcony',
 'May You Never Be Alone',
 'Alleluia! Sing To Jesus!',
 'America The Beautiful']

In [41]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))